In [1]:
import matplotlib as mpl
mpl.use("pgf")
import numpy as np
mm_to_in = 1/25.4
pgf_with_pdflatex = {
    "pgf.texsystem": "pdflatex",
    "pgf.preamble": [
         r"\usepackage{mathtools}",
         ],
    "text.usetex": True,
    "figure.figsize": [190*mm_to_in, 3.0],
    "axes.labelsize": 10,
    "axes.labelpad": 5.0,
    "font.size": 12,
    "font.family": "serif",
    "legend.fontsize": 8,
    "xtick.labelsize": 9,
    "ytick.labelsize": 9,
    "lines.linewidth": 1.5,
    "xtick.major.size": 5,
    "xtick.major.width": 1,
    "xtick.minor.size": 2.5,
    "xtick.minor.width": 1,
    "ytick.major.size": 5,
    "ytick.major.width": 1,
    "ytick.minor.size": 2.5,
    "ytick.minor.width": 1,
}
mpl.rcParams.update(pgf_with_pdflatex)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.legend_handler import HandlerLine2D
from matplotlib.ticker import FormatStrFormatter
from cycler import cycler
from pathlib import Path

In [2]:
phi_1_files = [Path('pressure-traces')/p for p in [Path('Tc_700K_P0_2.1851_T0_353K_pressure.txt'), Path('Tc_733K_P0_1.8041_T0_353K_pressure.txt'), Path('Tc_774K_P0_1.4504_T0_353K_pressure.txt')]]
phi_2_files = [Path('pressure-traces')/p for p in [Path('Tc_698K_P0_0.7118_T0_373K_pressure.txt'), Path('Tc_720K_P0_0.7806_T0_373K_pressure.txt'), Path('Tc_782K_P0_0.7995_T0_413K_pressure.txt')]]

In [3]:
data = pd.read_csv('pressure-traces/complete-data-set.csv')

phi_200_15_bar = data[(data['Equivalence Ratio'] == 2.00) &
                      (np.isclose(data['Compressed Pressure (bar)'], 15, rtol=1E-2))]
phi_100_30_bar = data[(data['Equivalence Ratio'] == 1.00) &
                      (np.isclose(data['Compressed Pressure (bar)'], 30, rtol=1E-2))]

phi_1_temps = phi_100_30_bar['Compressed Temperature (K)'].iloc[[1, 3, 5]]
phi_1_comp_times = phi_100_30_bar['Compression Time (ms)'].iloc[[1, 3, 5]]
phi_1_comp_times.iloc[1] += 0.5
phi_2_temps = phi_200_15_bar['Compressed Temperature (K)'].iloc[[3, 5, 11]]
phi_2_comp_times = phi_200_15_bar['Compression Time (ms)'].iloc[[3, 5, 11]]

In [4]:
fig, (ax_phi_1, ax_phi_2) = plt.subplots(ncols=2)

for i, f in enumerate(phi_1_files):
    data = np.loadtxt(str(f))
    max_idx = np.argmax(data[:, 1])
    ax_phi_1.plot(data[:max_idx, 0]*1000 - phi_1_comp_times.iloc[i], data[:max_idx, 1], label='$T_C = {}$ K'.format(phi_1_temps.iloc[i]))
    
nrdata = np.loadtxt('pressure-traces/NR_Tc_700K_P0_2.1851_T0_353K_pressure.txt')
ax_phi_1.plot(nrdata[:, 0]*1000 - phi_1_comp_times.iloc[0] - 0.5, nrdata[:, 1], label='Non-Reactive')

ax_phi_1.minorticks_on()

ax_phi_1.set_xlim(-20, 15)
ax_phi_1.set_ylim(0, 60)
ax_phi_1.set_ylabel('Pressure, bar')
ax_phi_1.set_xlabel('Time, ms')
ax_phi_1.legend(loc='upper center')
ax_phi_1.text(0.75, 0.05, 'a) $\phi=1.0$', transform=ax_phi_1.transAxes)

for i, f in enumerate(phi_2_files):
    data = np.loadtxt(str(f))
    max_idx = np.argmax(data[:, 1])
    ax_phi_2.plot(data[:max_idx, 0]*1000 - phi_2_comp_times.iloc[i], data[:max_idx, 1], label='$T_C = {}$ K'.format(phi_2_temps.iloc[i]))

nrdata = np.loadtxt('pressure-traces/NR_Tc_698K_P0_0.7118_T0_373K_pressure.txt')
ax_phi_2.plot(nrdata[:, 0]*1000 - phi_2_comp_times.iloc[0] + 0.5, nrdata[:, 1], label='Non-Reactive')

ax_phi_2.minorticks_on()

ax_phi_2.set_xlim(-20, 40)
ax_phi_2.set_ylim(0, 25)
ax_phi_2.set_xlabel('Time, ms')
ax_phi_2.legend(loc='upper center')
ax_phi_2.text(0.75, 0.05, 'b) $\phi=2.0$', transform=ax_phi_2.transAxes)

In [5]:
fig.tight_layout()
fig.savefig('pressure-traces.pdf')